# EpiRust Propensity Score Matching Demo

This notebook demonstrates EpiRust's propensity score matching capabilities for causal inference in observational studies. We'll cover:

1. Propensity score estimation
2. Different matching algorithms
3. Balance assessment
4. Treatment effect estimation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from epirust.matching import PropensityMatcher
from epirust.stats import BalanceStats

# Set random seed and plotting style
np.random.seed(42)
plt.style.use('seaborn')
sns.set_palette("husl")

## Generate Synthetic Observational Data

Let's create a dataset with confounding by age, sex, and comorbidities:

In [ ]:
def generate_observational_data(n_samples=1000):
    # Generate covariates
    age = np.random.normal(50, 15, n_samples)
    sex = np.random.binomial(1, 0.5, n_samples)
    comorbidity_score = np.random.poisson(2, n_samples)
    
    # Generate treatment assignment (confounded)
    logit = -1 + 0.03 * age + 0.5 * sex + 0.3 * comorbidity_score
    p_treat = 1 / (1 + np.exp(-logit))
    treatment = np.random.binomial(1, p_treat)
    
    # Generate outcome (affected by treatment and confounders)
    outcome = 10 + 2 * treatment + 0.1 * age + sex + 0.5 * comorbidity_score + \
              np.random.normal(0, 2, n_samples)
    
    return pd.DataFrame({
        'age': age,
        'sex': sex,
        'comorbidity_score': comorbidity_score,
        'treatment': treatment,
        'outcome': outcome
    })

# Generate data
df = generate_observational_data()
print("Data summary:")
print(df.describe())

## Estimate Propensity Scores

We'll use logistic regression to estimate propensity scores:

In [ ]:
# Create matcher object
matcher = PropensityMatcher()

# Estimate propensity scores
covariates = ['age', 'sex', 'comorbidity_score']
ps_scores = matcher.estimate_propensity_scores(df, 'treatment', covariates)

# Plot propensity score distributions
plt.figure(figsize=(10, 6))
sns.kdeplot(data=df, x=ps_scores, hue='treatment', common_norm=False)
plt.xlabel('Propensity Score')
plt.title('Propensity Score Distribution by Treatment Group')
plt.show()

## Compare Matching Algorithms

Let's try different matching approaches:

In [ ]:
# Perform matching with different algorithms
algorithms = ['nearest', 'caliper', 'optimal']
matched_dfs = {}

for algo in algorithms:
    matched_df = matcher.match(
        df,
        'treatment',
        covariates,
        method=algo,
        caliper=0.2 if algo == 'caliper' else None
    )
    matched_dfs[algo] = matched_df
    
    print(f"\nMatching results for {algo}:")
    print(f"Original samples: {len(df)}")
    print(f"Matched samples: {len(matched_df)}")
    print(f"Matched pairs: {len(matched_df) // 2}")

## Assess Balance

Let's check the covariate balance before and after matching:

In [ ]:
def plot_balance(original_df, matched_df, covariates):
    balance_stats = BalanceStats()
    
    # Calculate standardized differences
    orig_diff = balance_stats.standardized_differences(original_df, 'treatment', covariates)
    matched_diff = balance_stats.standardized_differences(matched_df, 'treatment', covariates)
    
    # Plot
    plt.figure(figsize=(10, 6))
    x = np.arange(len(covariates))
    width = 0.35
    
    plt.bar(x - width/2, orig_diff, width, label='Before matching')
    plt.bar(x + width/2, matched_diff, width, label='After matching')
    
    plt.axhline(y=0.1, color='r', linestyle='--', alpha=0.5)
    plt.axhline(y=-0.1, color='r', linestyle='--', alpha=0.5)
    
    plt.xlabel('Covariates')
    plt.ylabel('Standardized Difference')
    plt.title('Covariate Balance')
    plt.xticks(x, covariates, rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.show()

# Plot balance for each matching algorithm
for algo, matched_df in matched_dfs.items():
    print(f"\nBalance assessment for {algo} matching:")
    plot_balance(df, matched_df, covariates)

## Estimate Treatment Effects

Now let's estimate the average treatment effect on the treated (ATT):

In [ ]:
def estimate_att(matched_df):
    treated = matched_df[matched_df['treatment'] == 1]['outcome']
    control = matched_df[matched_df['treatment'] == 0]['outcome']
    
    att = np.mean(treated) - np.mean(control)
    se = np.sqrt(np.var(treated) / len(treated) + np.var(control) / len(control))
    ci = (att - 1.96 * se, att + 1.96 * se)
    
    return att, se, ci

# Calculate ATT for each matching method
results = []
for algo, matched_df in matched_dfs.items():
    att, se, ci = estimate_att(matched_df)
    results.append({
        'Method': algo,
        'ATT': att,
        'SE': se,
        'CI_lower': ci[0],
        'CI_upper': ci[1]
    })

results_df = pd.DataFrame(results)
print("Treatment Effect Estimates:")
print(results_df.to_string(index=False))

## Sensitivity Analysis

Let's perform sensitivity analysis for unmeasured confounding:

In [ ]:
# Perform Rosenbaum bounds sensitivity analysis
gamma_values = np.linspace(1, 2, 20)
sensitivity_results = matcher.sensitivity_analysis(
    matched_dfs['optimal'],
    'outcome',
    gamma_values
)

# Plot sensitivity bounds
plt.figure(figsize=(10, 6))
plt.plot(gamma_values, sensitivity_results['lower_bound'], label='Lower bound')
plt.plot(gamma_values, sensitivity_results['upper_bound'], label='Upper bound')
plt.axhline(y=0, color='r', linestyle='--', alpha=0.5)
plt.xlabel('Gamma (sensitivity parameter)')
plt.ylabel('Treatment Effect')
plt.title('Sensitivity Analysis for Unmeasured Confounding')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Conclusion

This notebook demonstrated EpiRust's propensity score matching capabilities:

1. Different matching algorithms (nearest neighbor, caliper, optimal)
2. Comprehensive balance assessment
3. Treatment effect estimation
4. Sensitivity analysis

Key findings:
- Optimal matching achieved the best covariate balance
- Treatment effect estimates were consistent across methods
- Results are moderately robust to unmeasured confounding